# Analyse de données financières : Détection d'anomalies dans un porteuille d'actifs

In [ ]:
import quandl
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

from sklearn.ensemble import IsolationForest

# instruction pour plotter dans notebook
%matplotlib inline

# connection à quandl via l'api key
# quandl.ApiConfig.api_key = " 3CipYF1Y3fzjvDgg7E2n"

In [ ]:
#lecture du fichier contenant les métadata des dataset de la base de données, directement importé de quandl
codes = pd.read_csv("https://www.quandl.com/api/v3/datasets.csv?database_code=EURONEXT&per_page=100&sort_by=id&page=1&api_key=3CipYF1Y3fzjvDgg7E2n",index_col="id",usecols=[0,1,2])

#Constitution du différents codes de dataset 
# le .4 signifie que l'on veut garder juste la quatrième colonne
codes['codes'] = codes.iloc[:,1]+'/'+codes.iloc[:,0]+'.4'

In [ ]:
# importation des données à l'aide d'une fonction pour faire le tout une bonne foi

def get(tickers, startdate, enddate):
    def data(ticker):
        return (quandl.get(ticker, start_date=startdate, end_date=enddate, authtoken="3CipYF1Y3fzjvDgg7E2n"))
    datas = map (data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))

In [ ]:
# récupération de la colonne des codes puis conversion en list
tickers = codes.iloc[0:40,2].values.tolist()

In [ ]:
all_data = get(tickers, datetime.datetime(2010, 1, 1), datetime.datetime(2015, 1, 1))

In [ ]:
all_data.unstack(level = -1)

## Rendements

 DataFrame.diff(periods=1, axis=0)[source]

    1st discrete difference of object
    Parameters:	

    periods : int, default 1

        Periods to shift for forming difference

    axis : {0 or ‘index’, 1 or ‘columns’}, default 0

        Take difference over rows (0) or columns (1).

    Returns:	

    diffed : DataFrame

In [ ]:
# 

rendments = all_data/all_data.diff(1) - 1

## Moyennes mobiles

In [ ]:
# Calculate the moving average
moving_avg = all_data.rolling(window=40).mean()

## Volatilités

In [ ]:
# volatilité

# Define the minumum of periods to consider 
min_periods = 75 

# Calculate the volatility
vol = daily_pct_change.rolling(min_periods).std() * np.sqrt(min_periods) 

# Plot the volatility
vol.plot(figsize=(10, 8))

# Show the plot
plt.show()

# Detection d'anomalies

In [ ]:
# df_2 est de dataframe qui contient les données dont on doit détecter les anomalies
# Applications de l'isolation forest sur les données de rendements ou de volatilités
clf = IsolationForest(n_estimator = 100, bootstrap = True)

# fit de l'estimateur
clf.fit(df_2)

# the anomaly score of the input samples. the lower the more abnormal.
scores_pred = clf.decision_fonction(df_2)

In [ ]:
# Détection d'anomalies sur les actifs

